In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
import os
import requests

tpdb_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['TPDB_API_KEY']}",
}

def get_scene_tpdb_id(stash_scene):
    # Extract TPDB ID from stash_ids
    tpdb_id = None
    for stash_id in stash_scene.get('stash_ids', []):
        if stash_id['endpoint'] == 'https://theporndb.net/graphql':
            tpdb_id = stash_id['stash_id']
            break
    return tpdb_id

def get_tpdb_scene_data(tpdb_id):
    url = f"https://api.theporndb.net/scenes/{tpdb_id}"
    response = requests.get(url, headers=tpdb_headers)
    return response.json()

# Function to convert seconds to MM:SS or HH:MM:SS format
def format_time(seconds):
    minutes, secs = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
    else:
        return f"{minutes:02d}:{secs:02d}"


In [ ]:
marker_source_tpdb = "Marker Source: TPDB"
marker_source_skier = "Marker Source: Skier AI"
marker_source_manual = "Marker Source: Manual"

marker_source_tpdb_tag = stash.find_tag(marker_source_tpdb)
marker_source_skier_tag = stash.find_tag(marker_source_skier)
marker_source_manual_tag = stash.find_tag(marker_source_manual)

# Processing single scene

In [ ]:
scene_id = 12661

stash_scene = stash.find_scene(scene_id)
print(stash_scene['title'])

tpdb_id = get_scene_tpdb_id(stash_scene)
print(tpdb_id)

tpdb_scene_data = get_tpdb_scene_data(tpdb_id)
print(tpdb_scene_data['data']['title'])


In [ ]:
tpdb_markers = tpdb_scene_data['data']['markers']
tpdb_markers

for marker in tpdb_markers:
    print(marker)
    

In [ ]:
scene_markers = stash_scene['scene_markers']

for marker in scene_markers:
    print({
        'title': marker['title'],
        'seconds': marker['seconds'],
        'primary_tag': marker['primary_tag']['name'],
        'tags': [tag['name'] for tag in marker['tags']]
    })

In [ ]:
# Convert Stash markers to a set of (title, seconds) tuples for easy comparison
stash_markers_set = set((marker['title'], marker['seconds']) for marker in scene_markers)

# Function to convert seconds to MM:SS or HH:MM:SS format
def format_time(seconds):
    minutes, secs = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
    else:
        return f"{minutes:02d}:{secs:02d}"

# Find markers in TPDB that are not in Stash
new_markers = []
print("Markers in TPDB but not in Stash:")
for tpdb_marker in tpdb_markers:
    title = tpdb_marker['title']
    seconds = tpdb_marker['start_time']
    formatted_time = format_time(seconds)
    if (title, seconds) not in stash_markers_set:
        print(f"- {title} at {formatted_time}")
        # Get the tag ID from Stash
        tag = stash.find_tag(title)
        if tag:
            tpdb_marker['tag_id'] = tag['id']
        else:
            print(f"Warning: Tag '{title}' not found in Stash")
        new_markers.append(tpdb_marker)

print(f"\nTotal new markers to add: {len(new_markers)}")


In [ ]:
for new_marker in new_markers:
    stash.create_scene_marker({
        "scene_id": scene_id,
        "title": new_marker['title'],
        "primary_tag_id": new_marker['tag_id'],
        "seconds": new_marker['start_time'],
        "tag_ids": [marker_source_tpdb_tag['id']],
    })


# Batch operation

In [ ]:
markers_tpdb_missing_tag = stash.find_tag("Markers: TPDB: Missing")
markers_tpdb_missing_tag

In [ ]:
markers_tpdb_saved_tag = stash.find_tag("Markers: TPDB: Saved")
markers_tpdb_saved_tag

In [ ]:
scenes_with_tpdb_id = stash.find_scenes({
    "stash_id_endpoint": {"endpoint": "https://theporndb.net/graphql", "modifier": "NOT_NULL" },
    "tags": {
        "value": [],
        "modifier": "INCLUDES_ALL",
        "excludes": [markers_tpdb_missing_tag['id'], markers_tpdb_saved_tag['id']]
    }
}, { "per_page": 250, "page": 1 })


In [ ]:
scenes_updated_with_markers = 0
for scene in scenes_with_tpdb_id:
    tpdb_id = get_scene_tpdb_id(scene)
    tpdb_scene_data = get_tpdb_scene_data(tpdb_id)
    tpdb_markers = tpdb_scene_data['data']['markers']
    scene_markers = scene['scene_markers']

    # Convert Stash markers to a set of (title, seconds) tuples for easy comparison
    stash_markers_set = set((marker['title'], marker['seconds']) for marker in scene_markers)

    # If TPDB does not have any markers, add markers_tpdb_missing_tag to the scene
    if not tpdb_markers:
        # Re-read the scene from Stash to get the most up-to-date data
        updated_scene = stash.find_scene(scene['id'])
        # Ensure we're using the most recent tag IDs
        current_tag_ids = [tag['id'] for tag in updated_scene['tags']]
        # Add the missing tag ID if it's not already present
        if markers_tpdb_missing_tag['id'] not in current_tag_ids:
            current_tag_ids.append(markers_tpdb_missing_tag['id'])
        stash.update_scene({
            "id": scene['id'],
            "tag_ids": current_tag_ids
        })
        print(f"Added 'Markers: TPDB: Missing' tag to scene ID: {scene['id']}, Title: {scene['title']}")
        continue  # Skip to the next scene


    # Find markers in TPDB that are not in Stash
    new_markers = []
    for tpdb_marker in tpdb_markers:
        title = tpdb_marker['title']
        seconds = tpdb_marker['start_time']
        if (title, seconds) not in stash_markers_set:
            # Get the tag ID from Stash
            tag = stash.find_tag(title)
            if tag:
                tpdb_marker['tag_id'] = tag['id']
                new_markers.append(tpdb_marker)
            else:
                print(f"Warning: Tag '{title}' not found in Stash")

    if new_markers:
        scenes_updated_with_markers += 1
        print(f"Scene ID: {scene['id']}, Title: {scene['title']}, Total new markers to add: {len(new_markers)}")

        for new_marker in new_markers:
            stash.create_scene_marker({
                "scene_id": scene['id'],
                "title": new_marker['title'],
                "primary_tag_id": new_marker['tag_id'],
                "seconds": new_marker['start_time'],
                "tag_ids": [marker_source_tpdb_tag['id']],
            })
        
        updated_scene = stash.find_scene(scene['id'])
        # Ensure we're using the most recent tag IDs
        current_tag_ids = [tag['id'] for tag in updated_scene['tags']]
        # Add the saved tag ID if it's not already present
        if markers_tpdb_saved_tag['id'] not in current_tag_ids:
            current_tag_ids.append(markers_tpdb_saved_tag['id'])
        stash.update_scene({
            "id": scene['id'],
            "tag_ids": current_tag_ids
        })

print(f"Scenes updated with markers: {scenes_updated_with_markers}")


# Foo

In [ ]:
all_tpdb_scene_markers = stash.find_scene_markers({
    "tags": {
        "value": [marker_source_tpdb_tag['id']],
        "modifier": "INCLUDES_ALL"
    }
})

unique_scene_ids = set()
for marker in all_tpdb_scene_markers:
    unique_scene_ids.add(marker['scene']['id'])

for scene_id in unique_scene_ids:
    scene = stash.find_scene(scene_id)
    tags = scene['tags']
    if not any(tag['id'] == markers_tpdb_missing_tag['id'] for tag in tags):
        update_tag_ids = [tag['id'] for tag in tags]
        update_tag_ids.append(markers_tpdb_saved_tag['id'])
        stash.update_scene({
            "id": scene_id,
            "tag_ids": update_tag_ids
        })

    

